<a href="https://colab.research.google.com/github/Eddie-Sun/GPT_from_scratch/blob/main/backpropagation_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline